In [1]:
# On linux, you probably will need this, but on mac things may be faster without it
# import sys
# sys.path.append('/Applications/Julia-1.3.app/Contents/Resources/julia/bin')

from julia.api import Julia
jl = Julia(compiled_modules=False)

from julia.DMUStudent.HW4 import mc, gw
from julia.RLInterface import *
from julia.Base import deepcopy

import gym
from gym.spaces import Box, Discrete

from julia.DMUStudent import evaluate
from julia.Base import convert, Function, Float64

In [2]:
from stable_baselines import DQN
from stable_baselines.common.vec_env import DummyVecEnv



/Users/maggie/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/maggie/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/maggie/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/maggie/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework

In [3]:
from julia.DMUStudent import evaluate
from julia.Base import convert, Function, Float64


In [4]:
help(gym.Env)

Help on class Env in module gym.core:

class Env(builtins.object)
 |  The main OpenAI Gym class. It encapsulates an environment with
 |  arbitrary behind-the-scenes dynamics. An environment can be
 |  partially or fully observed.
 |  
 |  The main API methods that users of this class need to know are:
 |  
 |      step
 |      reset
 |      render
 |      close
 |      seed
 |  
 |  And set the following attributes:
 |  
 |      action_space: The Space object corresponding to valid actions
 |      observation_space: The Space object corresponding to valid observations
 |      reward_range: A tuple corresponding to the min and max possible rewards
 |  
 |  Note: a default reward range set to [-inf,+inf] already exists. Set it if you want a narrower range.
 |  
 |  The methods are accessed publicly as "step", "reset", etc.. The
 |  non-underscored versions are wrapper methods to which we may add
 |  functionality over time.
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |      

In [4]:
class MCEnv(gym.Env):
    def __init__(self, jlenv):
        self.jlenv = jlenv

    def step(self, a):
        # if you use a continuous action space it will come in a vector
        # assert len(a) == 1
        # return step_b(self.jlenv, a[0])

        # decode discrete actions
        jl_act = [-1.0, 0.0, 1.0][a]
        s, r, done, info = step_b(self.jlenv, jl_act)
        pyinfo = {"episode": True}
        return s, r, done, pyinfo

    def reset(self):
        return reset_b(self.jlenv)

    @property
    def observation_space(self):
        return Box(low=-1.0, high=1.0, shape=(2,))

    @property
    def action_space(self):
        # if you want a continuous action space, use a box
        # return Box(low=-1.0, high=1.0, shape=(1,))
        return Discrete(3)

In [5]:
dqn = DQN('MlpPolicy',  DummyVecEnv([lambda: MCEnv(deepcopy(mc))]), verbose=1, exploration_fraction=0.1)

print("getting ready to learn...")
dqn.learn(total_timesteps=10)

def policy_function(s):
    act, st = dqn.predict(s)
    jl_act = [-1.0, 0.0, 1.0][act] # careful that this matches action decoding above!!
    return convert(Float64, jl_act)
    
evaluate(convert(Function, policy_function), "hw4", n_episodes=100)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.


/Users/maggie/anaconda3/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Instructions for updating:
Use tf.cast instead.
getting ready to learn...


0.0